# The main cell of this notebook will run an experiment. 

# It is assumed that Mind Monitor is already streaming OSC to the correct IP and port. 

# When experiment is complete, data will be automatically saved to MongoDB.

In [1]:
ip = "192.168.50.20"
port = 5000

import datetime

# SET EXPERIMENT PARAMETERS HERE
conditionList=['eyes open', 'eyes closed']
segmentDuration=10   # seconds
cycles=30    # total duration will be (len(conditionList) * segmentDuration * cycles + delayTime)
posture='sitting'
experimentDescription ='testing and debugging code'
delayTime = 5   # Integer countdown until start of experiment. Must be at least 2 seconds to allow OSC server to start. 
#exp_id ='test_sitting_2'
exp_id = 'test_'+posture+'_'+datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")  # ensures unique exp_id

import time
from collections import namedtuple
import sched
import threading
import numpy as np
import queue
import pyttsx3
from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import ThreadingOSCUDPServer

import pymongo
client = pymongo.MongoClient()
db = client.implementation_database
#db.collection_names()   # ['RawData', 'Experiments']


class ExperimentManager:    
    def __init__(self, exp_id, experimentDescription, conditionList, segmentDuration, cycles, posture, delayTimer=15):
        # For Experiment entry to MongoDB
        self.experimentParams = {'exp_id':exp_id,
                         'experimentDescription':experimentDescription,
                         'conditionList':conditionList,
                         'cycles':cycles,
                         'segmentDuration':segmentDuration,
                         'numSegments':len(conditionList)*cycles,
                         'segment_id_list':[str(exp_id)+'_'+str(segNum) for segNum in range(len(conditionList)*cycles)],
                         'segment_condition_list':conditionList * cycles,
                         'posture':posture,
                         'delayTimer':delayTimer}
        # Create experiment document in MongoDB
        db.Experiments.insert_one(self.experimentParams)
        
    def compute_TTS_strings(self):
        #compute TTS strings
        TTS_gap_between_utterances = 2
        
        testTimes = []
        testTimes.extend([i for i in range(-self.experimentParams['delayTimer'], 0, TTS_gap_between_utterances)])
        for segNum in range(self.experimentParams['numSegments']):
            testTimes.extend([i + (segNum * self.experimentParams['segmentDuration']) for i in range(0,self.experimentParams['segmentDuration'],TTS_gap_between_utterances)])
        testTimes.extend([self.experimentParams['numSegments'] * self.experimentParams['segmentDuration']])
        
        TTS_stringsList = []
        TTS_stringsList.extend([str(i) for i in range(-self.experimentParams['delayTimer'],0,TTS_gap_between_utterances)])
        for segNum in range(self.experimentParams['numSegments']):
            TTS_stringsList.extend([str(i) if i != 0 else self.experimentParams['segment_condition_list'][segNum] for i in range(0,self.experimentParams['segmentDuration'],TTS_gap_between_utterances)])
        TTS_stringsList.append('Protocol Complete')
        return TTS_stringsList, testTimes
            

expLengthSeconds = len(conditionList) * segmentDuration * cycles

EXPERIMENTER = ExperimentManager(exp_id=exp_id, 
                                 experimentDescription=experimentDescription, 
                                 conditionList=conditionList, 
                                 segmentDuration=segmentDuration, 
                                 cycles=cycles, 
                                 posture=posture, 
                                 delayTimer=delayTime)

# Start TTS process
TTS_list, test_times = EXPERIMENTER.compute_TTS_strings()
print('TTS_String_list',TTS_list)
print('TTS_Times',test_times)
print('Length:',len(TTS_list))

########
# TTS (Text-To-Speech) thread

def print_extra(a='default'):
            print("MAIN LOOP "+ str(time.time()-START_TIME_main) + a)
            
class TTS_thread(threading.Thread):
    def __init__(self, test_strings, test_times, START_TIME):
        super(TTS_thread, self).__init__()
        self.test_strings = test_strings
        self.test_times = [t - test_times[0] for t in test_times]
        self.START_TIME = START_TIME
        
    def run(self):
        s = sched.scheduler(time.time, time.sleep)

        def print_time(START_TIME, a='default'):
            print("From print_time "+ str(time.time()-START_TIME)+' '+str(a))
            TTS.say(a)
            TTS.runAndWait()

        def print_some_times(test_strings, test_times, START_TIME):
            print(time.time() - START_TIME)
            for i, timemark in enumerate(test_times):
                s.enter(timemark, 1, print_time, kwargs={'START_TIME':START_TIME, 'a': test_strings[i]})
            s.run()
            print(time.time() - START_TIME)

        TTS = pyttsx3.init()
        print_some_times(self.test_strings, self.test_times, START_TIME=self.START_TIME)


##############
# OSC Server and message handlers


test_q = queue.Queue()

def default_handler(address: str,*args):
    test_q.put((address, (time.time()-START_TIME_main, *args)))

q_acc = queue.Queue()
q_gyro = queue.Queue()
q_eeg = queue.Queue()
q_flags = queue.Queue()

def default_handler(address: str,*args):
    test_q.put({(address, (time.time()-START_TIME_main, *args))})
    
def acc_handler(address: str,*args):
    q_acc.put((time.time()-START_TIME_main, *args))

def gyro_handler(address: str,*args):
    q_gyro.put((time.time()-START_TIME_main, *args))
    
def contact_handler(address: str,*args):
    if args[0] != 1:
        q_flags.put((time.time()-START_TIME_main,'flag_No_Contact'))

def hsi_handler(address: str,*args):
    if ((args[0]+args[1]+args[2]+args[3])!=4):
        for hsi_i, hsi_name in zip(range(4), ['hsi_tp9','hsi_af7','hsi_af8','hsi_tp10']):
            if args[hsi_i]!=1:
                q_flags.put((time.time()-START_TIME_main,'flag_'+hsi_name))

def blink_handler(address: str,*args):
    if args[0] == 1:
        q_flags.put((time.time()-START_TIME_main,'flag_blink'))
            
def jaw_clench_handler(address: str,*args):
    if args[0] == 1:
        q_flags.put((time.time()-START_TIME_main,'flag_jaw_clench'))

def eeg_handler(address: str,*args):
    q_eeg.put((time.time()-START_TIME_main, *args))

dispatcher = Dispatcher()
dispatcher.map("/muse/eeg", eeg_handler)  # 256Hz - raw electrode signal ['TP9','AF7','AF8','TP10']
dispatcher.map("/muse/acc", acc_handler)    # 52Hz - accelerometer, 3 args for X,Y,Z
dispatcher.map("/muse/gyro", gyro_handler)  # 52Hz - gyroscope, 3 args for X,Y,Z
dispatcher.map("/muse/elements/touching_forehead", contact_handler)  # 10Hz - 1 argument
dispatcher.map("/muse/elements/horseshoe", hsi_handler)              # 10Hz - 4 args for ['TP9','AF7','AF8','TP10']
dispatcher.map("/muse/elements/blink", blink_handler)            # 10Hz (if true)
dispatcher.map("/muse/elements/jaw_clench", jaw_clench_handler)  # 10Hz (if true)
dispatcher.set_default_handler(default_handler)

server = ThreadingOSCUDPServer((ip, port), dispatcher)

def stopStream():
    server.server_close()  ### Stops the server. Will give an error, but it will work.
    
def printStopIntent():
    print('Trying to close server '+str(time.time() - START_TIME_main + 0.05))

class stopThread(threading.Thread):
    def __init__(self, delayTime, expLengthSeconds):
        super(stopThread, self).__init__()
        self.delayTime = delayTime
        self.expLengthSeconds = expLengthSeconds
    def run(self):
        s_main = sched.scheduler(time.time, time.sleep)
        s_main.enter(self.delayTime + self.expLengthSeconds + 0.05, 1, stopStream)
        s_main.enter(self.delayTime + self.expLengthSeconds -0.1+ 0.05, 1, printStopIntent)
        s_main.run()
        print('Stopped server '+str(time.time() - START_TIME_main))

START_TIME_main = time.time() + delayTime  # Start the experiment clock.
        
t1 = TTS_thread(TTS_list, test_times, START_TIME_main)
t1.start()
print('Started TTS thread')

t2 = stopThread(delayTime, expLengthSeconds)
t2.start()
print('Started server countdown thread')

try:
    server.serve_forever() # Blocks forever, until it is closed in a way that returns an error.
except:
    list_eeg = []
    while not q_eeg.empty():
        list_eeg.append(q_eeg.get())
    list_acc = []
    while not q_acc.empty():
        list_acc.append(q_acc.get())
    list_gyro = []
    while not q_gyro.empty():
        list_gyro.append(q_gyro.get())
    list_flags = []
    while not q_flags.empty():
        list_flags.append(q_flags.get())

    time.sleep(2) # Allow other threads time to finish.
    segDr = EXPERIMENTER.experimentParams['segmentDuration']
        
    eegArrays=[]
    k=4
    fieldList = ['t','tp9','af7','af8','tp10']
    for i1,i2 in zip(range(0,expLengthSeconds,segDr),range(segDr,expLengthSeconds+1,segDr)):
        eegArrays.append([{key:list_eeg[p][i] for i,key in zip(range(k+1),fieldList)} for p in range(len(list_eeg)) if list_eeg[p][0] > i1 and list_eeg[p][0] <= i2 ])
    print(len(eegArrays),'EEG segments')
    for l in eegArrays:
        print(len(l))

    accArrays=[]
    k=3
    fieldList = ['t','acc_x','acc_y','acc_z']
    for i1,i2 in zip(range(0,expLengthSeconds,segDr),range(segDr,expLengthSeconds+1,segDr)):
        accArrays.append([{key:list_acc[p][i] for i,key in zip(range(k+1),fieldList)} for p in range(len(list_acc)) if list_acc[p][0] > i1 and list_acc[p][0] <= i2 ])
    print(len(accArrays),'ACC segments')
    for l in accArrays:
        print(len(l))

    gyroArrays=[]
    k=3
    fieldList = ['t','gyro_x','gyro_y','gyro_z']
    for i1,i2 in zip(range(0,expLengthSeconds,segDr),range(segDr,expLengthSeconds+1,segDr)):
        gyroArrays.append([{key:list_gyro[p][i] for i,key in zip(range(k+1),fieldList)} for p in range(len(list_gyro)) if list_gyro[p][0] > i1 and list_gyro[p][0] <= i2 ])
    print(len(gyroArrays),'GYRO segments')
    for l in gyroArrays:
        print(len(l))

    flagsArrays=[]
    k=1
    fieldList = ['t','flag']
    for i1,i2 in zip(range(0,expLengthSeconds,segDr),range(segDr,expLengthSeconds+1,segDr)):
        flagsArrays.append([{key:list_flags[p][i] for i,key in zip(range(k+1),fieldList)} for p in range(len(list_flags)) if list_flags[p][0] > i1 and list_flags[p][0] <= i2 ])
    print(len(flagsArrays),'FLAG segments')
    for l in flagsArrays:
        print(len(l))
        
    for seg in range(EXPERIMENTER.experimentParams['numSegments']):
        entryDict = {'segment_id':EXPERIMENTER.experimentParams['segment_id_list'][seg], 
                     'segment_order_num':seg,
                     'exp_id':EXPERIMENTER.experimentParams['exp_id'],  
                     'condition_type':EXPERIMENTER.experimentParams['segment_condition_list'][seg],
                     'duration_seconds':EXPERIMENTER.experimentParams['segmentDuration'], 
                     'numFlags':len(flagsArrays[seg]), 
                     'flagsData':flagsArrays[seg],
                     'numPointsEEG':len(eegArrays[seg]), 
                     'eegData':eegArrays[seg],
                     'numPointsACC':len(accArrays[seg]),
                     'accData':accArrays[seg],
                     'numPointsGYRO':len(gyroArrays[seg]),
                     'gyroData':gyroArrays[seg]}
        db.RawData.insert_one(entryDict)   
        print('Segment data saved to database')


TTS_String_list ['-5', '-3', '-1', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', '2', '4', '6', '8', 'eyes closed', '2', '4', '6', '8', 'eyes open', 

From print_time 232.17649364471436 2
From print_time 234.18128085136414 4
From print_time 236.18507289886475 6
From print_time 238.1839644908905 8
From print_time 240.18179297447205 eyes open
From print_time 242.47316241264343 2
From print_time 244.1784164905548 4
From print_time 246.17701530456543 6
From print_time 248.18167209625244 8
From print_time 250.18450498580933 eyes closed
From print_time 252.17142939567566 2
From print_time 254.17589783668518 4
From print_time 256.17992401123047 6
From print_time 258.1835868358612 8
From print_time 260.1827414035797 eyes open
From print_time 262.1736922264099 2
From print_time 264.1752920150757 4
From print_time 266.1813027858734 6
From print_time 268.1723852157593 8
From print_time 270.17709136009216 eyes closed
From print_time 272.17832136154175 2
From print_time 274.18347120285034 4
From print_time 276.18016934394836 6
From print_time 278.17941522598267 8
From print_time 280.18073534965515 eyes open
From print_time 282.1823182106018 2
Fro

Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment data saved to database
Segment 

In [6]:
import pymongo
client = pymongo.MongoClient()
db = client.implementation_database
#db.collection_names()   # ['RawData', 'Experiments']


In [2]:
db.Experiments.find().count()

3

In [3]:
db.RawData.find().count()

68

In [8]:
# This syntax will delete records.
#DATABASE.COLLECTION.delete_many(FILTER_CRITERIA) 

In [7]:
docs = db.Experiments.find({})
for doc in docs:
    print(doc)
    print('\n')

{'_id': ObjectId('60f61d6cd64ad42a08e5f20c'), 'exp_id': 'test_sitting_0', 'experimentDescription': 'testing and debugging code', 'conditionList': ['eyes open', 'eyes closed'], 'cycles': 2, 'segmentDuration': 10, 'numSegments': 4, 'segment_id_list': ['test_sitting_0_0', 'test_sitting_0_1', 'test_sitting_0_2', 'test_sitting_0_3'], 'segment_condition_list': ['eyes open', 'eyes closed', 'eyes open', 'eyes closed'], 'posture': 'sitting', 'delayTimer': 5}


{'_id': ObjectId('60f61e05d64ad4090c20a855'), 'exp_id': 'test_sitting_1', 'experimentDescription': 'testing and debugging code', 'conditionList': ['eyes open', 'eyes closed'], 'cycles': 2, 'segmentDuration': 10, 'numSegments': 4, 'segment_id_list': ['test_sitting_1_0', 'test_sitting_1_1', 'test_sitting_1_2', 'test_sitting_1_3'], 'segment_condition_list': ['eyes open', 'eyes closed', 'eyes open', 'eyes closed'], 'posture': 'sitting', 'delayTimer': 5}


{'_id': ObjectId('60f61eb4d64ad440bc6dd5bd'), 'exp_id': 'test_sitting_2', 'experimentDes